Our input is a CSV file consisting of ~187k lines, each line representing a candidate URL. Our goal is to clean the input so that each line is a unique, parseable URL with no query parameters or fragments. We're not interested in filtering these URLs based on their response status; we only want to normalize the structure so that they can be ready for future HTTP requests.

In [17]:
import pandas as pd
df = pd.read_csv('data/raw/urls.csv')
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187010 entries, 0 to 187009
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   Url     187010 non-null  object
dtypes: object(1)
memory usage: 1.4+ MB


Our dataframe has a single column, labelled 'Url' with 187,010 rows of string objects. Let's view the first 10 rows

In [18]:
from pandas import option_context
# temporarily increase the column width for visibility
with option_context('display.max_colwidth', 400):
    print(df.head(10))

                                                                            Url
0                                                                   rehabs.com/
1   rehabs.com/10-famous-models-who-have-battled-drug-addiction-and-alcoholism/
2  rehabs.com/14-famous-child-actors-who-battled-drug-addiction-and-alcoholism/
3            rehabs.com/20-celebrities-who-actively-support-addiction-recovery/
4                   rehabs.com/3-very-different-stories-of-court-ordered-rehab/
5                 rehabs.com/4-barriers-plaguing-on-demand-addiction-treatment/
6                      rehabs.com/5-celebrities-in-rehab-for-different-reasons/
7                 rehabs.com/5-celebrity-stoners-who-later-renounced-marijuana/
8                   rehabs.com/5-honest-reasons-we-lie-to-ourselves-and-others/
9             rehabs.com/5-inspirational-quotes-to-help-you-overcome-addiction/


What we should immediately notice about the URL structure is that they don't begin with either a scheme or subdomain, and that they end with a trailing slash. Let's see if this holds true for the entire dataset.

In [22]:
prefix_count = len(df[df['Url'].str.startswith('rehabs.com')])
suffix_count = len(df[df['Url'].str.endswith('/')])
print(f'URLs with rehabs.com prefix: {prefix_count}')
print(f'URLs with trailing slash: {suffix_count}')

URLs with rehabs.com prefix: 187010
URLs with trailing slash: 151352


All of the URLs begin with rehabs.com, but some don't end with a trailing slash. Let's identify what portion of these contain query strings or fragments

In [74]:
missing_trailing_slash = df[~df['Url'].str.endswith('/')]
regex_pat = '(\?|\#)'
# We can use Series.str.extract in conjuction with Dataframe.groupby() to group records by presence of substring.
extract = missing_trailing_slash.Url.str.extract(regex_pat, expand=False)
t = missing_trailing_slash.groupby(s).size().reset_index(name="Count")
t['Percentage'] = round(100 * t['Count']  / t['Count'].sum(), 2)
t.head()

,Url,Count,Percentage
0,#,210,0.59
1,?,35259,99.41


So most of the URLs with missing trailing slashes are due to the presence of query parameters. We'll have to split those records out using the `urlsplit()` function from `urllib.parse`

In [78]:
from urllib.parse import urlsplit

# first let's lowercase the URLs and strip any whitespace from the ends
df = df['Url'].apply(lambda x: x.lower().strip())

# next we'll prefix strings with protocol so urlsplit can parse the components.
df['Url'] = df['Url'].apply(lambda x: 'https://' + x if not x.startswith('https://') else x)

# create new columns for URL components
df['Scheme'],df['Netloc'],df['Path'],df['Query'],df['Fragment'] = zip(*df['Url'].map(urlsplit))
df.head()

,Url,Scheme,Netloc,Path,Query,Fragment
0,https://rehabs.com/,https,rehabs.com,/,,
1,https://rehabs.com/10-famous-models-who-have-battled-drug-addiction-and-alcoholism/,https,rehabs.com,/10-famous-models-who-have-battled-drug-addiction-and-alcoholism/,,
2,https://rehabs.com/14-famous-child-actors-who-battled-drug-addiction-and-alcoholism/,https,rehabs.com,/14-famous-child-actors-who-battled-drug-addiction-and-alcoholism/,,
3,https://rehabs.com/20-celebrities-who-actively-support-addiction-recovery/,https,rehabs.com,/20-celebrities-who-actively-support-addiction-recovery/,,
4,https://rehabs.com/3-very-different-stories-of-court-ordered-rehab/,https,rehabs.com,/3-very-different-stories-of-court-ordered-rehab/,,


Now that the dataframe is split, we can start deduplication.

In [79]:
# drop the Url, Query, and Fragment columns so that we're just left with Scheme, Netloc and Path.
# if we were concerned that some of the URLs had a `www` subdomain, we could drop Netloc too and add the right one in.
df = df.drop(columns=['Url', 'Fragment','Query'])

# if a trailing slash does not exist at the end of the Path, we add it on
df['Path'] = df.Path.apply(lambda s: s if s.endswith('/') else s + '/')

# remove duplicate records
df = df.drop_duplicates()

print(len(df))

151541


We've reduced our dataset to 152K urls, all unique and prefixed with the correct scheme. We now recreate our URL column and write it out to CSV

In [81]:
# recreate Url column
df['Url'] = df['Scheme'] + '://' + df['Netloc'] + df['Path']
df.to_csv('data/normalized/urls.csv', columns=['Url'], index=False)